# Design ideas

## Specifying LOS distributions
Could use eval in following way:

- just allow distributions available in numpy
- only allow numeric args
- we prepend on the rv generator variable

Can always have external tools that make it easy to create config files. The
files themselves should not be using global variables. But... right now I've got
parameters of erlang LOS distributions as global vars. So, it would be nice if 
distributions could use global variables. Could have an 'los_param_vars' section that
could then be used in the distributions section. I kind of like that.

In [1]:
import numpy as np
import pandas as pd
from numpy.random import default_rng
import numpy.random
import yaml
import json
from functools import partial
import copy

In [3]:
rg = default_rng(1)

In [21]:
obs_dist_str = 'exponential(5.0)'

In [22]:
eval('1 + 2')

3

In [23]:
obs_dist_rg_str = f'rg.{obs_dist_str}'
obs_dist_rg_str

'rg.exponential(5.0)'

In [24]:
eval(obs_dist_rg_str)

5.365145131862694

In [10]:
# Compile the expression

code = compile(obs_dist_rg_str, "<string>", "eval")
print(code)

<code object <module> at 0x7f91dc5f9a80, file "<string>", line 1>


In [11]:
dir(code)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'co_argcount',
 'co_cellvars',
 'co_code',
 'co_consts',
 'co_filename',
 'co_firstlineno',
 'co_flags',
 'co_freevars',
 'co_kwonlyargcount',
 'co_lnotab',
 'co_name',
 'co_names',
 'co_nlocals',
 'co_posonlyargcount',
 'co_stacksize',
 'co_varnames',
 'replace']

In [12]:
for n in code.co_names:
    print(n)

rg
exponential


In [13]:
for n in code.co_consts:
    print(n)

5.0


In [14]:
obs_dist_str = 'exponential(mean_los)'
obs_dist_rg_str = f'rg.{obs_dist_str}'
obs_dist_rg_str
co_obs_dist = compile(obs_dist_rg_str, "<string>", "eval")

for n in co_obs_dist.co_names:
    print(n)
    
for n in co_obs_dist.co_consts:
    print(n)
    
for n in co_obs_dist.co_freevars:
    print(n)

rg
exponential
mean_los


In [5]:
los_params_str = """
los_params:
    mean_los_csect: 1.0
    mean_los_ldr: 12.0
    mean_los_obs: 2.4
    mean_los_pp_c: 72.0
    mean_los_pp_noc: 48.0
    num_erlang_stages_csect: 4
    num_erlang_stages_ldr: 2
    num_erlang_stages_obs: 1
    num_erlang_stages_pp: 8"""

los_params = yaml.safe_load(los_params_str)
los_params

{'los_params': {'mean_los_csect': 1.0,
  'mean_los_ldr': 12.0,
  'mean_los_obs': 2.4,
  'mean_los_pp_c': 72.0,
  'mean_los_pp_noc': 48.0,
  'num_erlang_stages_csect': 4,
  'num_erlang_stages_ldr': 2,
  'num_erlang_stages_obs': 1,
  'num_erlang_stages_pp': 8}}

In [6]:
los_dists_str = """
los_distributions:
    RAND_SPONT_REG:
        OBS: gamma(num_erlang_stages_obs, mean_los_obs / num_erlang_stages_obs)
        LDR: gamma(num_erlang_stages_ldr, mean_los_ldr / num_erlang_stages_ldr)
        PP:  gamma(num_erlang_stages_pp, mean_los_pp_noc / num_erlang_stages_pp)
    RAND_SPONT_CSECT:
        OBS: gamma(num_erlang_stages_obs, mean_los_obs / num_erlang_stages_obs)
        LDR: gamma(num_erlang_stages_ldr, mean_los_ldr / num_erlang_stages_ldr)
        LDR: gamma(num_erlang_stages_csect, mean_los_csect / num_erlang_stages_csect)
        PP:  gamma(num_erlang_stages_pp, mean_los_pp_c / num_erlang_stages_pp)"""

los_dists = yaml.safe_load(los_dists_str)
los_dists

{'los_distributions': {'RAND_SPONT_REG': {'OBS': 'gamma(num_erlang_stages_obs, mean_los_obs / num_erlang_stages_obs)',
   'LDR': 'gamma(num_erlang_stages_ldr, mean_los_ldr / num_erlang_stages_ldr)',
   'PP': 'gamma(num_erlang_stages_pp, mean_los_pp_noc / num_erlang_stages_pp)'},
  'RAND_SPONT_CSECT': {'OBS': 'gamma(num_erlang_stages_obs, mean_los_obs / num_erlang_stages_obs)',
   'LDR': 'gamma(num_erlang_stages_csect, mean_los_csect / num_erlang_stages_csect)',
   'PP': 'gamma(num_erlang_stages_pp, mean_los_pp_c / num_erlang_stages_pp)'}}}

In [17]:
ALLOWED_DIST_LIST = ['beta', 'binomial', 'chisquare', 'exponential', 'gamma',
                    'geometric', 'hypergeometric', 'laplace', 'logistic', 'lognormal',
                    'multinomial', 'negative_binomial', 'normal', 'pareto',
                    'poisson', 'triangular', 'uniform','weibull', 'zipf']
{

    k: v for k, v in numpy.random.__dict__.items() if k in ALLOWED_DIST_LIST

}

{'beta': <function RandomState.beta>,
 'binomial': <function RandomState.binomial>,
 'chisquare': <function RandomState.chisquare>,
 'exponential': <function RandomState.exponential>,
 'gamma': <function RandomState.gamma>,
 'geometric': <function RandomState.geometric>,
 'hypergeometric': <function RandomState.hypergeometric>,
 'laplace': <function RandomState.laplace>,
 'logistic': <function RandomState.logistic>,
 'lognormal': <function RandomState.lognormal>,
 'multinomial': <function RandomState.multinomial>,
 'negative_binomial': <function RandomState.negative_binomial>,
 'normal': <function RandomState.normal>,
 'pareto': <function RandomState.pareto>,
 'poisson': <function RandomState.poisson>,
 'triangular': <function RandomState.triangular>,
 'uniform': <function RandomState.uniform>,
 'weibull': <function RandomState.weibull>,
 'zipf': <function RandomState.zipf>}

In [11]:
co_test_dist_str = los_dists['los_distributions']['RAND_SPONT_REG']['OBS']
co_test_dist_str

'gamma(num_erlang_stages_obs, mean_los_obs / num_erlang_stages_obs)'

In [15]:
test_dist_rg_str = f'rg.{co_test_dist_str}'
co_test_dist = compile(test_dist_rg_str, "<string>", "eval")

print("**names**")
for n in co_test_dist.co_names:
    print(n)
print("**varnames**")
for n in co_test_dist.co_varnames:
    print(n)
print("**consts**")   
for n in co_test_dist.co_consts:
    print(n)
print("**freevars**")    
for n in co_test_dist.co_freevars:
    print(n)

**names**
rg
gamma
num_erlang_stages_obs
mean_los_obs
**varnames**
**consts**
freevars


In [18]:
for n in co_test_dist.co_names:
    okname = n in ALLOWED_DIST_LIST or n == 'rg' or n in los_params['los_params']
    print(n, okname)
    
    if n in los_params['los_params']

rg True
gamma True
num_erlang_stages_obs True
mean_los_obs True


In [26]:
los_global_params = los_params['los_params']
los_global_params

{'mean_los_csect': 1.0,
 'mean_los_ldr': 12.0,
 'mean_los_obs': 2.4,
 'mean_los_pp_c': 72.0,
 'mean_los_pp_noc': 48.0,
 'num_erlang_stages_csect': 4,
 'num_erlang_stages_ldr': 2,
 'num_erlang_stages_obs': 1,
 'num_erlang_stages_pp': 8}

I think we need to just step through the global params:
 - check if in each of the distributions
 - if we find it, replace it with its value (gotten from the dict).
 
Above idea is tricky/tedious since we have to search the nested dict. Easier approach is to use json to dump the dict to a string, do all the S&R and read back into a dict.

In [32]:
print(los_dists)

#for param in los_global_params:
param = 'num_erlang_stages_obs'

los_dists_str_json = json.dumps(los_dists)
los_dists_str_json = los_dists_str_json.replace(param, str(los_global_params[param]))

los_dists = json.loads(los_dists_str_json)
print(los_dists)   

{'los_distributions': {'RAND_SPONT_REG': {'OBS': 'gamma(num_erlang_stages_obs, mean_los_obs / num_erlang_stages_obs)', 'LDR': 'gamma(num_erlang_stages_ldr, mean_los_ldr / num_erlang_stages_ldr)', 'PP': 'gamma(num_erlang_stages_pp, mean_los_pp_noc / num_erlang_stages_pp)'}, 'RAND_SPONT_CSECT': {'OBS': 'gamma(num_erlang_stages_obs, mean_los_obs / num_erlang_stages_obs)', 'LDR': 'gamma(num_erlang_stages_csect, mean_los_csect / num_erlang_stages_csect)', 'PP': 'gamma(num_erlang_stages_pp, mean_los_pp_c / num_erlang_stages_pp)'}}}
{'los_distributions': {'RAND_SPONT_REG': {'OBS': 'gamma(1, mean_los_obs / 1)', 'LDR': 'gamma(num_erlang_stages_ldr, mean_los_ldr / num_erlang_stages_ldr)', 'PP': 'gamma(num_erlang_stages_pp, mean_los_pp_noc / num_erlang_stages_pp)'}, 'RAND_SPONT_CSECT': {'OBS': 'gamma(1, mean_los_obs / 1)', 'LDR': 'gamma(num_erlang_stages_csect, mean_los_csect / num_erlang_stages_csect)', 'PP': 'gamma(num_erlang_stages_pp, mean_los_pp_c / num_erlang_stages_pp)'}}}


In [33]:
print(los_dists)

los_dists_str_json = json.dumps(los_dists)
for param in los_global_params:
    los_dists_str_json = los_dists_str_json.replace(param, str(los_global_params[param]))

los_dists = json.loads(los_dists_str_json)
print(los_dists)   

{'los_distributions': {'RAND_SPONT_REG': {'OBS': 'gamma(1, mean_los_obs / 1)', 'LDR': 'gamma(num_erlang_stages_ldr, mean_los_ldr / num_erlang_stages_ldr)', 'PP': 'gamma(num_erlang_stages_pp, mean_los_pp_noc / num_erlang_stages_pp)'}, 'RAND_SPONT_CSECT': {'OBS': 'gamma(1, mean_los_obs / 1)', 'LDR': 'gamma(num_erlang_stages_csect, mean_los_csect / num_erlang_stages_csect)', 'PP': 'gamma(num_erlang_stages_pp, mean_los_pp_c / num_erlang_stages_pp)'}}}
{'los_distributions': {'RAND_SPONT_REG': {'OBS': 'gamma(1, 2.4 / 1)', 'LDR': 'gamma(2, 12.0 / 2)', 'PP': 'gamma(8, 48.0 / 8)'}, 'RAND_SPONT_CSECT': {'OBS': 'gamma(1, 2.4 / 1)', 'LDR': 'gamma(4, 1.0 / 4)', 'PP': 'gamma(8, 72.0 / 8)'}}}


Now, let's get one dist and create a partial function based on it eval()'d.

In [34]:
co_test_dist_str = los_dists['los_distributions']['RAND_SPONT_REG']['OBS']
co_test_dist_str

'gamma(1, 2.4 / 1)'

In [37]:
eval(f'rg.{co_test_dist_str}')

12.901048494259504

In [38]:
co_test_dist_str_list = [los_dists['los_distributions']['RAND_SPONT_REG']['OBS']]
co_test_dist_str_list

['gamma(1, 2.4 / 1)']

In [52]:
def get_args_and_kwargs(*args, **kwargs):
    return args, kwargs

def convert_str_to_args_and_kwargs(s):
    return eval(s.replace(s[:s.find('(')], 'get_args_and_kwargs'))

def convert_str_to_func_name(s):
    return s[:s.find('(')]

for s in co_test_dist_str_list:
    func_name = convert_str_to_func_name(s)
    args, kwargs = convert_str_to_args_and_kwargs(s)
    print(func_name, args, kwargs)

gamma (1, 2.4) {}


In [40]:
args

(1, 2.4)

In [41]:
partial(rg.gamma, args)

functools.partial(<built-in method gamma of numpy.random._generator.Generator object at 0x7f148694b580>, (1, 2.4))

In [45]:
new_gamma = partial(rg.gamma, *args)

In [48]:
new_gamma()

0.07131226070946209

In [49]:
new_gamma2 = partial(eval('rg.gamma'), *args)

In [51]:
new_gamma()

2.499500194434012

Ok, let's try to put it all together using the los_dists dict

In [4]:
los_dists_str = """
    RAND_SPONT_REG:
        OBS: gamma(num_erlang_stages_obs, mean_los_obs / num_erlang_stages_obs)
        LDR: gamma(num_erlang_stages_ldr, mean_los_ldr / num_erlang_stages_ldr)
        PP:  gamma(num_erlang_stages_pp, mean_los_pp_noc / num_erlang_stages_pp)
    RAND_SPONT_CSECT:
        OBS: gamma(num_erlang_stages_obs, mean_los_obs / num_erlang_stages_obs)
        LDR: gamma(num_erlang_stages_ldr, mean_los_ldr / num_erlang_stages_ldr)
        LDR: gamma(num_erlang_stages_csect, mean_los_csect / num_erlang_stages_csect)
        PP:  gamma(num_erlang_stages_pp, mean_los_pp_c / num_erlang_stages_pp)"""

los_dists = yaml.safe_load(los_dists_str)
los_dists

{'RAND_SPONT_REG': {'OBS': 'gamma(num_erlang_stages_obs, mean_los_obs / num_erlang_stages_obs)',
  'LDR': 'gamma(num_erlang_stages_ldr, mean_los_ldr / num_erlang_stages_ldr)',
  'PP': 'gamma(num_erlang_stages_pp, mean_los_pp_noc / num_erlang_stages_pp)'},
 'RAND_SPONT_CSECT': {'OBS': 'gamma(num_erlang_stages_obs, mean_los_obs / num_erlang_stages_obs)',
  'LDR': 'gamma(num_erlang_stages_csect, mean_los_csect / num_erlang_stages_csect)',
  'PP': 'gamma(num_erlang_stages_pp, mean_los_pp_c / num_erlang_stages_pp)'}}

In [6]:
los_params_str = """
mean_los_csect: 1.0
mean_los_ldr: 12.0
mean_los_obs: 2.4
mean_los_pp_c: 72.0
mean_los_pp_noc: 48.0
num_erlang_stages_csect: 4
num_erlang_stages_ldr: 2
num_erlang_stages_obs: 1
num_erlang_stages_pp: 8"""

los_params = yaml.safe_load(los_params_str)
los_params

{'mean_los_csect': 1.0,
 'mean_los_ldr': 12.0,
 'mean_los_obs': 2.4,
 'mean_los_pp_c': 72.0,
 'mean_los_pp_noc': 48.0,
 'num_erlang_stages_csect': 4,
 'num_erlang_stages_ldr': 2,
 'num_erlang_stages_obs': 1,
 'num_erlang_stages_pp': 8}

In [10]:
ALLOWED_LOS_DIST_LIST = ['beta', 'binomial', 'chisquare', 'exponential', 'gamma',
                    'geometric', 'hypergeometric', 'laplace', 'logistic', 'lognormal',
                    'multinomial', 'negative_binomial', 'normal', 'pareto',
                    'poisson', 'triangular', 'uniform','weibull', 'zipf']

def get_args_and_kwargs(*args, **kwargs):
    return args, kwargs

def convert_str_to_args_and_kwargs(s):
    return eval(s.replace(s[:s.find('(')], 'get_args_and_kwargs'))

def convert_str_to_func_name(s):
    return s[:s.find('(')]

los_dists_str_json = json.dumps(los_dists)
for param in los_params:
    los_dists_str_json = los_dists_str_json.replace(param, str(los_params[param]))

los_dists_instantiated = json.loads(los_dists_str_json)

los_dists_partials = copy.deepcopy(los_dists_instantiated)
for key_pat_type in los_dists_partials:
    for key_unit, raw_dist_str in los_dists_partials[key_pat_type].items():
        func_name = convert_str_to_func_name(raw_dist_str)
        print(func_name)
        # Check for valid func name
        if func_name in ALLOWED_LOS_DIST_LIST:
            args, kwargs = convert_str_to_args_and_kwargs(raw_dist_str)
            print(args, kwargs)
            partial_dist_func = partial(eval(f'rg.{func_name}'), *args)
            los_dists_partials[key_pat_type][key_unit] = partial_dist_func
        else:
            raise NameError(f"The use of '{func_name}' is not allowed")

print(los_dists_partials)

gamma
(1, 2.4) {}
gamma
(2, 6.0) {}
gamma
(8, 6.0) {}
gamma
(1, 2.4) {}
gamma
(4, 0.25) {}
gamma
(8, 9.0) {}
{'RAND_SPONT_REG': {'OBS': functools.partial(<built-in method gamma of numpy.random._generator.Generator object at 0x7f2e89ef34a0>, 1, 2.4), 'LDR': functools.partial(<built-in method gamma of numpy.random._generator.Generator object at 0x7f2e89ef34a0>, 2, 6.0), 'PP': functools.partial(<built-in method gamma of numpy.random._generator.Generator object at 0x7f2e89ef34a0>, 8, 6.0)}, 'RAND_SPONT_CSECT': {'OBS': functools.partial(<built-in method gamma of numpy.random._generator.Generator object at 0x7f2e89ef34a0>, 1, 2.4), 'LDR': functools.partial(<built-in method gamma of numpy.random._generator.Generator object at 0x7f2e89ef34a0>, 4, 0.25), 'PP': functools.partial(<built-in method gamma of numpy.random._generator.Generator object at 0x7f2e89ef34a0>, 8, 9.0)}}


In [59]:
los_dists_partials

{'los_distributions': {'RAND_SPONT_REG': {'OBS': functools.partial(<built-in method gamma of numpy.random._generator.Generator object at 0x7f148694b580>, 1, 2.4), 'LDR': functools.partial(<built-in method gamma of numpy.random._generator.Generator object at 0x7f148694b580>, 2, 6.0), 'PP': functools.partial(<built-in method gamma of numpy.random._generator.Generator object at 0x7f148694b580>, 8, 6.0)}, 'RAND_SPONT_CSECT': {'OBS': functools.partial(<built-in method gamma of numpy.random._generator.Generator object at 0x7f148694b580>, 1, 2.4), 'LDR': functools.partial(<built-in method gamma of numpy.random._generator.Generator object at 0x7f148694b580>, 4, 0.25), 'PP': functools.partial(<built-in method gamma of numpy.random._generator.Generator object at 0x7f148694b580>, 8, 9.0)}}}

## Graph operations

In [11]:
import networkx as nx

In [12]:
route_graph = nx.DiGraph()

In [13]:
routes_str = """routes:
  1:
    edges:
    - from: ENTRY
      id: ENTRY_OBS
      to: OBS
    - from: OBS
      id: OBS_LDR
      to: LDR
    - from: LDR
      id: LDR_PP
      to: PP
    - from: PP
      id: PP_EXIT
      to: EXIT
    id: 1
  2:
    edges:
    - from: ENTRY
      id: ENTRY_OBS
      to: OBS
    - from: OBS
      id: OBS_LDR
      to: LDR
    - from: LDR
      id: LDR_CSECT
      to: CSECT
    - from: CSECT
      id: CSECT_pp
      to: PP
    - from: PP
      id: PP_EXIT
      to: EXIT
    id: 2"""

In [14]:
routes = yaml.safe_load(routes_str)
routes

{'routes': {1: {'edges': [{'from': 'ENTRY', 'id': 'ENTRY_OBS', 'to': 'OBS'},
    {'from': 'OBS', 'id': 'OBS_LDR', 'to': 'LDR'},
    {'from': 'LDR', 'id': 'LDR_PP', 'to': 'PP'},
    {'from': 'PP', 'id': 'PP_EXIT', 'to': 'EXIT'}],
   'id': 1},
  2: {'edges': [{'from': 'ENTRY', 'id': 'ENTRY_OBS', 'to': 'OBS'},
    {'from': 'OBS', 'id': 'OBS_LDR', 'to': 'LDR'},
    {'from': 'LDR', 'id': 'LDR_CSECT', 'to': 'CSECT'},
    {'from': 'CSECT', 'id': 'CSECT_pp', 'to': 'PP'},
    {'from': 'PP', 'id': 'PP_EXIT', 'to': 'EXIT'}],
   'id': 2}}}

In [19]:
locations_str = """
  ENTRY:
    capacity: 1000
    name: ENTRY
  OBS:
    capacity: 100
    name: OBS
  LDR:
    capacity: 16
    name: LDR
  CSECT:
    capacity: 1000
    name: CSECT
  PP:
    capacity: 36
    name: PP
  EXIT:
    capacity: 1000
    name: EXIT"""

In [20]:
locations = yaml.safe_load(locations_str)
locations

{'ENTRY': {'capacity': 1000, 'name': 'ENTRY'},
 'OBS': {'capacity': 100, 'name': 'OBS'},
 'LDR': {'capacity': 16, 'name': 'LDR'},
 'CSECT': {'capacity': 1000, 'name': 'CSECT'},
 'PP': {'capacity': 36, 'name': 'PP'},
 'EXIT': {'capacity': 1000, 'name': 'EXIT'}}

In [21]:
for loc_name, location in locations.items():
    print(loc_name)
    print(location)
    route_graph.add_node(loc_name, 
                         planned_los=0.0, actual_los=0.0, blocked_duration=0.0,
                         name=location['name'])

ENTRY
{'capacity': 1000, 'name': 'ENTRY'}
OBS
{'capacity': 100, 'name': 'OBS'}
LDR
{'capacity': 16, 'name': 'LDR'}
CSECT
{'capacity': 1000, 'name': 'CSECT'}
PP
{'capacity': 36, 'name': 'PP'}
EXIT
{'capacity': 1000, 'name': 'EXIT'}


In [24]:
route_graph.nodes['OBS']['planned_los'] = 5.5

In [25]:
for node, data in route_graph.nodes(data=True):
    print(node)
    print(data)

ENTRY
{'planned_los': 0.0, 'actual_los': 0.0, 'blocked_duration': 0.0, 'name': 'ENTRY'}
OBS
{'planned_los': 5.5, 'actual_los': 0.0, 'blocked_duration': 0.0, 'name': 'OBS'}
LDR
{'planned_los': 0.0, 'actual_los': 0.0, 'blocked_duration': 0.0, 'name': 'LDR'}
CSECT
{'planned_los': 0.0, 'actual_los': 0.0, 'blocked_duration': 0.0, 'name': 'CSECT'}
PP
{'planned_los': 0.0, 'actual_los': 0.0, 'blocked_duration': 0.0, 'name': 'PP'}
EXIT
{'planned_los': 0.0, 'actual_los': 0.0, 'blocked_duration': 0.0, 'name': 'EXIT'}


## Simulation calendar

In [2]:
start_date = '2022-01-02'

In [3]:
start_date_ts = pd.Timestamp(start_date)
start_date_ts

Timestamp('2022-01-02 00:00:00')

In [8]:
start_date_ts.weekday()

6

In [4]:
env_now = 245.6

pd.to_timedelta(env_now, unit='h')

Timedelta('10 days 05:36:00')

In [5]:
now_calendar_time = start_date_ts + pd.to_timedelta(env_now, unit='h')
now_calendar_time

Timestamp('2022-01-12 05:36:00')

In [6]:
pd.Timedelta(now_calendar_time - start_date_ts, unit='h') / pd.to_timedelta(1, unit='h')

245.6

In [7]:
now_calendar_time.weekday()

2

In [5]:
start_date = '1970-01-05'

In [6]:
unix_epoch_ts = pd.Timestamp(start_date)
print(unix_epoch_ts)
print(unix_epoch_ts.weekday())

1970-01-05 00:00:00
0


## Scheduled cases

[2025-07-23] Ended up dumping the following idea in favor of a "long" format file that specifies dow, time, number scheduled. The time parameter is in simulation time units. The dow parameter is 'mon', 'tue', etc.

In [9]:
!ls

design_ideas.ipynb  input     mm_output   obflowsim_explainer.ipynb
exp1_run.sh	    mm_input  new_exp.sh  output


In [5]:
filename = 'input/sched_c.txt'
sched_c_arrivals = np.loadtxt(filename, dtype=int)

In [6]:
sched_c_arrivals

array([[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]])

In [7]:
sched_c_arrivals.shape

(7, 24)

In [8]:
pd.to_timedelta(1, unit='w') / pd.to_timedelta(1, unit='h')

168.0

In [9]:
arrival_epochs = [(d, h, sched_c_arrivals[d, h]) for d in range(6) 
                              for h in range(23) if sched_c_arrivals[d, h] > 0]
arrival_epochs

[(0, 6, 1),
 (0, 7, 1),
 (0, 8, 1),
 (2, 6, 1),
 (2, 14, 1),
 (2, 15, 1),
 (3, 6, 1),
 (3, 7, 1),
 (3, 8, 1),
 (3, 16, 1),
 (3, 17, 1),
 (3, 18, 1),
 (3, 20, 1),
 (4, 16, 1),
 (4, 17, 1),
 (4, 18, 1),
 (4, 20, 1)]

In [11]:
(day, hour, n) = (0, 6, 1)
pd.to_timedelta(day * 24 + hour, unit='h') / pd.to_timedelta(1, unit='h')

6.0

In [12]:
(day, hour, n) = (0, 6, 1)
pd.to_timedelta(day * 24 + hour, unit='h') / pd.to_timedelta(1, unit='m')

360.0